In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import cv2
from keras.models import load_model

In [2]:
#load data

train = pd.read_csv("sign_mnist_train.csv")
test = pd.read_csv("sign_mnist_test.csv")

a_train = np.array(train.drop(['label'], axis=1))
b_train = np.array(train['label'])

a_test = np.array(test.drop(['label'], axis=1))
b_test = np.array(test['label'])

In [3]:
#reshape data

a_train = a_train.reshape(a_train.shape[0], 28, 28, 1)
a_test = a_test.reshape(a_test.shape[0], 28, 28, 1)

#preprocess data

a_train = a_train / 255.0
a_test = a_test / 255.0

In [4]:
#build CNN model

CNNmodel = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(26, activation='softmax')
])

In [5]:
#compile and train the model

CNNmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
CNNmodel.summary()

CNNmodel.fit(a_train, b_train, epochs=10, batch_size=32, validation_data=(a_test, b_test))

CNNmodel.save('my_model.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                                 
 dense_1 (Dense)             (None, 26)                3354      
                                                                 
Total params: 696,026
Trainable params: 696,026
Non-trainable params: 0
__________________________________________________

In [6]:
#test the model

CNNmodel = load_model('my_model.h5')

loss, accuracy = CNNmodel.evaluate(a_test, b_test)

print('Test loss: ', loss)
print('Test Accuracy: ', accuracy)

225/225 [==============================] - 2s 7ms/step - loss: 0.9721 - accuracy: 0.8423
Test loss:  0.9720917344093323
Test Accuracy:  0.8423033952713013


In [7]:
# predict the model

from keras.models import load_model
CNNmodel = load_model('my_model.h5')
labels = ['A', 'B', 'C', 'D', 'E', 'F','G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (28,28))
    grayscale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    normalized_frame = grayscale_frame/255.0
    reshaped_frame = normalized_frame.reshape(1,28,28,1)
    return reshaped_frame

def predict(frame):
    preprocessed_frame = preprocess_frame(frame)
    prediction = CNNmodel.predict(preprocessed_frame)
    predicted_label = labels[np.argmax(prediction)]
    return predicted_label

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    cv2.imshow('Webcam', frame)
    predicted_label = predict(frame)
    cv2.putText(frame,str(predicted_label), (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Prediction', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 50ms/step


KeyboardInterrupt: 

: 

In [ ]:
#convert h5 file to tf lite format

import tensorflow as tf
from keras.models import load_model

model = load_model("my_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)

lite_model = converter.convert()

with open ("lite_model.tflite", "wb") as f:
    f.write(lite_model)

INFO:tensorflow:Assets written to: C:\Users\SANYAM~1\AppData\Local\Temp\tmp__vyo2nw\assets


INFO:tensorflow:Assets written to: C:\Users\SANYAM~1\AppData\Local\Temp\tmp__vyo2nw\assets
